In [1]:
%load_ext sparkmagic.magics
from dsx_core_utils import proxy_util,dsxhi_util
proxy_util.configure_proxy_livy()
dsxhi_util.list_livy_endpoints()

success configuring sparkmagic livy.
['https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1', 'https://qlawsdl001038a.ad.datalake.foc.zone:8443/gateway/dsx/livy/v1']


In [2]:
%%spark config
{"executorCores": 4, "numExecutors": 5, "executorMemory": "10g", 
 "driverMemory": "8g", "proxyUser": "jchen-", "driverCores": 1, 
 "conf": {"spark.yarn.appMasterEnv.THEANO_FLAGS": "base_compiledir=${PWD}/.theano"}}

In [3]:
%spark add -s servicingdata -k -l python -u https://qlawsbidlhe02a.ad.datalake.foc.zone:8445/gateway/dsx/livy2/v1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
21772,application_1590030838276_35322,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [4]:
%%spark

import pyspark
import os, sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when, lit, countDistinct, max, min, desc
from pyspark.sql import DataFrameStatFunctions as statFunc
import pyspark.sql.types as T
from functools import reduce

from os.path import expanduser, join, abspath
import time
import pandas as pd

from pyspark.ml import Pipeline, PipelineModel

spark = SparkSession.builder.getOrCreate()

In [5]:
%%spark

spark.version

'2.3.0.2.6.5.0-292'

In [6]:
%%spark

from pyspark.sql.functions import lower

csv_path = '/dev/projects/retention_models/csv_data/'
actual_path = '/dev/projects/retention_models/actual_value/'
monthly_path = '/dev/projects/retention_models/monthly_snapshot/'

past_path = '/dev/projects/retention_models/purchase_payoff/training/results/movermodel/'

In [8]:
%%spark
 
df_actual = spark.read.csv(csv_path + 'Payoff_segments_summary_20200610.csv', header=True)\
                        .withColumn('newloanpurpose', lower(col('NewLoanPurpose')))\
                        .withColumn('retentiontype', lower(col('PayoffRetentionType')))\
                        .select('ServicedLoanNumber', 'PaymentInFullDate', 'newloanpurpose', 'retentiontype')
    
df_list1 = spark.sql('''
select distinct servicecalendardate, ln_no
from data_science_sandbox.servicing_dbo_payoff_source_purchase_client_crdtrs 
where leftportfoliodate is null
and servicecalendardate between '2016-06-30' and '2018-11-30'
''')

## upload from DW table
## add more snapshot to update the current servicing loans payoff information

df_201812 = spark.read.parquet(monthly_path+'servicing_df_all_dec18_20190131.parquet').select('servicecalendardate', 'ln_no')
 
df_201901 = spark.read.parquet(monthly_path+'servicing_df_all_jan19_20190215.parquet').select('servicecalendardate', 'ln_no')
df_201902 = spark.read.parquet(monthly_path+'servicing_df_all_feb19_20190305.parquet').select('servicecalendardate', 'ln_no')
df_201903 = spark.read.parquet(monthly_path+'servicing_df_all_mar19_20190409.parquet').select('servicecalendardate', 'ln_no')
df_201904 = spark.read.parquet(monthly_path+'servicing_df_all_apr19_20190507.parquet').select('servicecalendardate', 'ln_no')
df_201905 = spark.read.parquet(monthly_path+'servicing_df_all_may19_20190610.parquet').select('servicecalendardate', 'ln_no')
df_201906 = spark.read.parquet(monthly_path+'servicing_df_all_june19_20190708.parquet').select('servicecalendardate', 'ln_no')
df_201907 = spark.read.parquet(monthly_path+'servicing_df_all_jul19_20190805.parquet').select('servicecalendardate', 'ln_no')
df_201908 = spark.read.parquet(monthly_path+'servicing_df_all_aug19_20191226.parquet').select('servicecalendardate', 'ln_no')
df_201909 = spark.read.parquet(monthly_path+'servicing_df_all_sep19_20191004.parquet').select('servicecalendardate', 'ln_no')
df_201910 = spark.read.parquet(monthly_path+'servicing_df_all_oct19_20191105.parquet').select('servicecalendardate', 'ln_no')
df_201911 = spark.read.parquet(monthly_path+'servicing_df_all_nov19_20191202.parquet').select('servicecalendardate', 'ln_no')
df_201912 = spark.read.parquet(monthly_path+'servicing_df_all_dec19_20200103.parquet').select('servicecalendardate', 'ln_no')

df_202001 = spark.read.parquet(monthly_path+'servicing_df_all_jan20_20200203.parquet').select('servicecalendardate', 'ln_no')
df_202002 = spark.read.parquet(monthly_path+'servicing_df_all_feb20_20200305.parquet').select('servicecalendardate', 'ln_no')
df_202003 = spark.read.parquet(monthly_path+'servicing_df_all_mar20_20200406.parquet').select('servicecalendardate', 'ln_no')
df_202004 = spark.read.parquet(monthly_path+'servicing_df_all_apr20_20200505.parquet').select('servicecalendardate', 'ln_no')
df_202005 = spark.read.parquet(monthly_path+'servicing_df_all_may20_20200610.parquet').select('servicecalendardate', 'ln_no')


    
## join all snapshot servicing lists
df_list = df_list1.union(df_201812)\
                    .union(df_201901).union(df_201902).union(df_201903)\
                    .union(df_201904).union(df_201905).union(df_201906)\
                    .union(df_201907).union(df_201908).union(df_201909)\
                    .union(df_201910).union(df_201911).union(df_201912)\
                    .union(df_202001).union(df_202002).union(df_202003)\
                    .union(df_202004).union(df_202005)
 

In [9]:
%%spark

## check the total count
df_list.groupBy('servicecalendardate').count().orderBy(desc('servicecalendardate')).show()

+-------------------+-------+
|servicecalendardate|  count|
+-------------------+-------+
|         2020-05-31|1889579|
|         2020-04-30|1899099|
|         2020-03-31|1867297|
|         2020-02-29|1846864|
|         2020-01-31|1824706|
|         2019-12-31|1802243|
|         2019-11-30|1807991|
|         2019-10-31|1787277|
|         2019-09-30|1763391|
|         2019-08-31|1856202|
|         2019-07-31|1839005|
|         2019-06-30|1822001|
|         2019-05-31|1806604|
|         2019-04-30|1788444|
|         2019-03-31|1770015|
|         2019-02-28|1752593|
|         2019-01-31|1738794|
|         2018-12-31|1726017|
|         2018-11-30|1713615|
|         2018-10-31|1699510|
+-------------------+-------+
only showing top 20 rows

In [10]:
%%spark

df_payoff = df_list.join(df_actual, df_actual.ServicedLoanNumber == df_list.ln_no, how='left')


In [11]:
%%spark

## create flags

def build_targets(df):
    
    # Purchase Payoff
    expr_purpayoff = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')) & (col('newloanpurpose')=='purchase'), 1.)\
                        .otherwise(0.)
    expr_purpayoff_3_6 = when((col('PaymentInFullDate')>col('enddt_3')) & (col('PaymentInFullDate')<=col('enddt_6')) & (col('newloanpurpose')=='purchase'), 1.)\
                            .otherwise(0.)
    expr_purpayoff_6_12 = when((col('PaymentInFullDate')>col('enddt_6')) & (col('PaymentInFullDate')<=col('enddt_12')) & (col('newloanpurpose')=='purchase'), 1.)\
                            .otherwise(0.)
    expr_purpayoff_all = when((col('PaymentInFullDate')>col('servicecalendardate')) & (col('newloanpurpose')=='purchase'), 1.)\
                            .otherwise(0.)
    
    # Refi Payoff
    expr_refipayoff = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')) & (col('newloanpurpose')!='purchase'), 1.)\
                        .otherwise(0.)
    expr_refipayoff_all = when((col('PaymentInFullDate')>col('servicecalendardate')) & (col('newloanpurpose')!='purchase'), 1.)\
                            .otherwise(0.)
    
    # Purchase/Refi Retained
    expr_purretain_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')) 
                             & (col('newloanpurpose')=='purchase') & (col('retentiontype')=='retained'), 1.)\
                            .otherwise(0.)
    
    expr_refiretain_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')) 
                             & (col('newloanpurpose')!='purchase') & (col('retentiontype')=='retained'), 1.)\
                            .otherwise(0.)
    
    # Payoff
    expr_payoff = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')), 1.)\
                        .otherwise(0.)
    expr_payoff_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')), 1.)\
                        .otherwise(0.)
    expr_payoff_all = when(col('PaymentInFullDate')>col('servicecalendardate'), 1.)\
                        .otherwise(0.)
    
    # Retained
    expr_retention = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_3')) & (col('retentiontype')=='retained'), 1.)\
                        .otherwise(0.)
    expr_retention_3_6 = when((col('PaymentInFullDate')>col('enddt_3')) & (col('PaymentInFullDate')<=col('enddt_6')) & (col('retentiontype')=='retained'), 1.)\
                            .otherwise(0.)
    expr_retention_6_12 = when((col('PaymentInFullDate')>col('enddt_6')) & (col('PaymentInFullDate')<=col('enddt_12')) & (col('retentiontype')=='retained'), 1.)\
                            .otherwise(0.)
    expr_retention_24 = when((col('PaymentInFullDate')>col('startdt')) & (col('PaymentInFullDate')<=col('enddt_24')) & (col('retentiontype')=='retained'), 1.)\
                        .otherwise(0.)
    expr_retention_all = when((col('PaymentInFullDate')>col('servicecalendardate')) & (col('retentiontype')=='retained'), 1.)\
                        .otherwise(0.)
        
    ## the 05-31 snapshot predicts 06 propensity
    ## when calculate actual payoff, need to start from 06-01 to get the payoffs
    ## and in this logic, payoffs are counted from 07-01
    df_final = df.withColumn('startdt', F.add_months(col('servicecalendardate'), 1))\
                    .withColumn('enddt_3', F.add_months(col('servicecalendardate'), 4))\
                    .withColumn('enddt_6', F.add_months(col('servicecalendardate'), 7))\
                    .withColumn('enddt_12', F.add_months(col('servicecalendardate'), 13))\
                    .withColumn('enddt_24', F.add_months(col('servicecalendardate'), 25))\
                    .withColumn('purchasepayoff', expr_purpayoff)\
                    .withColumn('purchasepayoff_3_6', expr_purpayoff_3_6)\
                    .withColumn('purchasepayoff_6_12', expr_purpayoff_6_12)\
                    .withColumn('purchasepayoff_all', expr_purpayoff_all)\
                    .withColumn('refipayoff', expr_refipayoff)\
                    .withColumn('refipayoff_all', expr_refipayoff_all)\
                    .withColumn('purretain_24', expr_purretain_24)\
                    .withColumn('refiretain_24', expr_refiretain_24)\
                    .withColumn('payoff', expr_payoff)\
                    .withColumn('payoff_24', expr_payoff_24)\
                    .withColumn('payoff_all', expr_payoff_all)\
                    .withColumn('retained', expr_retention)\
                    .withColumn('retained_3_6', expr_retention_3_6)\
                    .withColumn('retained_6_12', expr_retention_6_12)\
                    .withColumn('retained_24', expr_retention_24)\
                    .withColumn('retained_all', expr_retention_all)\
                    .select('servicecalendardate', 'ln_no', 'PaymentInFullDate', 'newloanpurpose', 'retentiontype',
                            'purchasepayoff', 'purchasepayoff_3_6', 'purchasepayoff_6_12', 'purchasepayoff_all',
                            'refipayoff', 'refipayoff_all', 
                            'purretain_24', 'refiretain_24',
                            'payoff', 'payoff_24', 'payoff_all', 
                            'retained', 'retained_3_6', 'retained_6_12', 'retained_24', 'retained_all')
                        
    return df_final
 

In [12]:
%%spark

df_payoff.groupBy('servicecalendardate').count().sort(desc('servicecalendardate')).show()

+-------------------+-------+
|servicecalendardate|  count|
+-------------------+-------+
|         2020-05-31|1889579|
|         2020-04-30|1899099|
|         2020-03-31|1867297|
|         2020-02-29|1846864|
|         2020-01-31|1824706|
|         2019-12-31|1802243|
|         2019-11-30|1807991|
|         2019-10-31|1787277|
|         2019-09-30|1763391|
|         2019-08-31|1856202|
|         2019-07-31|1839005|
|         2019-06-30|1822001|
|         2019-05-31|1806604|
|         2019-04-30|1788444|
|         2019-03-31|1770015|
|         2019-02-28|1752593|
|         2019-01-31|1738794|
|         2018-12-31|1726017|
|         2018-11-30|1713615|
|         2018-10-31|1699510|
+-------------------+-------+
only showing top 20 rows

In [13]:
%%spark

df_final = build_targets(df_payoff)

df_final.groupBy('servicecalendardate').count().sort(desc('servicecalendardate')).show()

+-------------------+-------+
|servicecalendardate|  count|
+-------------------+-------+
|         2020-05-31|1889579|
|         2020-04-30|1899099|
|         2020-03-31|1867297|
|         2020-02-29|1846864|
|         2020-01-31|1824706|
|         2019-12-31|1802243|
|         2019-11-30|1807991|
|         2019-10-31|1787277|
|         2019-09-30|1763391|
|         2019-08-31|1856202|
|         2019-07-31|1839005|
|         2019-06-30|1822001|
|         2019-05-31|1806604|
|         2019-04-30|1788444|
|         2019-03-31|1770015|
|         2019-02-28|1752593|
|         2019-01-31|1738794|
|         2018-12-31|1726017|
|         2018-11-30|1713615|
|         2018-10-31|1699510|
+-------------------+-------+
only showing top 20 rows

In [14]:
%%spark

df_final.write.parquet(actual_path + 'Actual_payoff_20200610.parquet', mode='overwrite')


In [14]:
%%spark

df_final = spark.read.parquet(actual_path + 'Actual_payoff_20200505.parquet')

In [15]:
%%spark

df_final.groupBy('servicecalendardate').count().sort(desc('servicecalendardate')).show()

+-------------------+-------+
|servicecalendardate|  count|
+-------------------+-------+
|         2020-04-30|1899099|
|         2020-03-31|1867297|
|         2020-02-29|1846864|
|         2020-01-31|1824706|
|         2019-12-31|1802243|
|         2019-11-30|1807991|
|         2019-10-31|1787277|
|         2019-09-30|1763391|
|         2019-08-31|1856202|
|         2019-07-31|1839005|
|         2019-06-30|1822001|
|         2019-05-31|1806604|
|         2019-04-30|1788444|
|         2019-03-31|1770015|
|         2019-02-28|1752593|
|         2019-01-31|1738794|
|         2018-12-31|1726017|
|         2018-11-30|1713615|
|         2018-10-31|1699510|
|         2018-09-30|1683583|
+-------------------+-------+
only showing top 20 rows

In [ ]:
%%spark

